In [3]:
import os
import pandas as pd
import requests
import getpass
from bs4 import BeautifulSoup
import json
import time
from tqdm.auto import tqdm
from datetime import datetime

In [5]:
key = getpass.getpass()

········


In [6]:
class TFTRiotAPI:
    def __init__(self):
        pass
    
    def summoner_name(self, key, name):
        name = name.replace(' ', '%20')
        request = requests.get(f'https://br1.api.riotgames.com/tft/summoner/v1/summoners/by-name/{name}?api_key={key}')
        return eval(request.content)
    
    def match_list(self, key, puuid, n = 30):
        request = requests.get(f'https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?count={n}&api_key={key}')
        return eval(request.content)
    
    def match_info(self, key, matchid, unixtime = False):
        '''Return a dataframe with some infos from the match (matchid, datetime, game lenght(s), game variation,
        game version, number of the set).

        key = riot API-key
        matchid = The id from a specific match of team fight tatics(TFT)'''

        request = requests.get(f'https://americas.api.riotgames.com/tft/match/v1/matches/{matchid}?api_key={key}')
        df = pd.DataFrame(pd.json_normalize(eval(request.content)))
        df2 = df[['metadata.match_id', 'info.game_datetime', 'info.game_length', 'info.game_variation',
           'info.game_version', 'info.tft_set_number']]
        df2.columns = [i.replace('metadata.', '').replace('info.', '') for i in df2.columns]
        
        y = df2.copy()
        if unixtime == False:
            x = df2['game_datetime'].apply(lambda x: pd.to_datetime(datetime.utcfromtimestamp(int(x)/1000).strftime('%Y-%m-%d %H:%M:%S')))
            y = df2.copy()
            y['game_datetime'] = x

        return y
    
    def match_participants(self, key, matchid):
        '''Return a list with the puuid from the 8 participants of the match'''

        request = requests.get(f'https://americas.api.riotgames.com/tft/match/v1/matches/{matchid}?api_key={key}')
        df = pd.DataFrame(pd.json_normalize(eval(request.content)))
        return df.iloc[0,2]
    
    def match_results(self, key, matchid):
        request = requests.get(f'https://americas.api.riotgames.com/tft/match/v1/matches/{matchid}?api_key={key}')
        soup = BeautifulSoup(request.content)
        df_master = pd.DataFrame(columns = ['match_id', 'gold_left', 'last_round', 'level', 'placement', 'players_eliminated', 'puuid',
                                     'time_eliminated', 'total_damage_to_players', 'Set3_Blademaster', 'Blaster',
                                     'Set3_Brawler', 'Set3_Celestial', 'Chrono', 'Cybernetic', 'DarkStar',
                                     'Demolitionist', 'Infiltrator', 'ManaReaver', 'MechPilot',
                                     'Mercenary', 'Set3_Mystic', 'Protector', 'Rebel', 'Sniper',
                                     'SpacePirate', 'StarGuardian', 'Starship', 'Set3_Sorcerer', 
                                     'Valkyrie', 'Vanguard', 'Set3_Void', 'champion_1', 'champion_2', 'champion_3',
                                     'champion_4', 'champion_5', 'champion_6', 'champion_7', 'champion_8', 'champion_9',
                                     'champion_10'])

        df_match = pd.DataFrame(pd.json_normalize(eval(soup.text)))
        dict_players = df_match.iloc[0,-3]

        for i in range(8):
            info_player = pd.DataFrame([dict_players[i]])
            df_master.loc[i, 'match_id'] = df_match.loc[0, 'metadata.match_id']
            df_master.loc[i, 'gold_left'] = info_player.loc[0, 'gold_left']
            df_master.loc[i, 'last_round'] = info_player.loc[0, 'last_round']
            df_master.loc[i, 'level'] = info_player.loc[0, 'level']
            df_master.loc[i, 'placement'] = info_player.loc[0, 'placement']
            df_master.loc[i, 'players_eliminated'] = info_player.loc[0, 'players_eliminated']
            df_master.loc[i, 'puuid'] = info_player.loc[0, 'puuid']
            df_master.loc[i, 'time_eliminated'] = info_player.loc[0, 'time_eliminated']
            df_master.loc[i, 'total_damage_to_players'] = info_player.loc[0, 'total_damage_to_players']

            for j in range(len(info_player.iloc[0, -2])): ## nesta etapa ele adiciona os tipos e quantidade, possui um valor extra no dicio que causa um bug (TemplateTrait)
                intermediate = pd.DataFrame(info_player.iloc[0, -2])
                df_master.loc[i, intermediate['name'][j]] = intermediate['num_units'][j]

            for k in range(len(info_player.iloc[0, -1])):
                intermediate = pd.DataFrame(info_player.iloc[0, -1])        
                df_master.loc[i, 'champion_' + str(k+1)] = (intermediate['character_id'][k] + '_' + str(intermediate['tier'][k])).replace('TFT3_', '')

        #df_master.columns = ['match_id', 'gold_left', 'last_round', 'level', 'placement', 'players_eliminated', 'puuid',
        #                             'time_eliminated', 'total_damage_to_players', 'blademaster_num', 'blaster_num',
        #                             'brawler_num', 'celestial_num', 'chrono_num', 'cybernetic_num', 'dark_star_num',
        #                             'demolitionist_num', 'infiltrator_num', 'mana_reaver_num', 'mech_pilot_num',
        #                             'mercenary_num', 'mystic_num', 'protector_num', 'rebel_num', 'sniper_num',
        #                             'space_pirate_num', 'star_guardian_num', 'starship_num', 'sorcerer_num', 
        #                             'valkyrie_num', 'vanguard_num', 'void_num', 'champion_1', 'champion_2', 'champion_3',
        #                             'champion_4', 'champion_5', 'champion_6', 'champion_7', 'champion_8', 'champion_9',
        #                             'champion_10']
        return df_master

In [7]:
pd.set_option('display.max_columns', 999)

In [8]:
riot = TFTRiotAPI()

In [63]:
puuid_me = riot.summoner_name(key, 'Es mir egal')['puuid']
puuid_me

'K3mbUvb-2PPBom5fdP5FBdn-2CUsQqy4t2wh-dmw-e55tv0nVirYpqo-AISeib2MLctqZ63zorohUg'

In [64]:
match_list_master = []

In [8]:
match_list = riot.match_list(key, puuid_me)
match_list

['BR1_1932405958',
 'BR1_1932041516',
 'BR1_1931990138',
 'BR1_1931763481',
 'BR1_1930827974',
 'BR1_1930769792',
 'BR1_1930780175',
 'BR1_1930744707',
 'BR1_1929766171',
 'BR1_1929751730',
 'BR1_1929631786',
 'BR1_1928808166',
 'BR1_1928783006',
 'BR1_1927689028',
 'BR1_1927580618',
 'BR1_1927468990',
 'BR1_1927461391',
 'BR1_1926730533',
 'BR1_1926675398',
 'BR1_1926670564',
 'BR1_1926614700',
 'BR1_1926538340',
 'BR1_1926140313',
 'BR1_1926073659',
 'BR1_1926067196',
 'BR1_1926063272',
 'BR1_1925978166',
 'BR1_1925750238',
 'BR1_1925712925',
 'BR1_1925627632']

In [9]:
puuid_list_master = []

In [10]:
for i in tqdm(match_list):
    temp_list = riot.match_participants(key, i)
    for i in temp_list:
        puuid_list_master.append(i)

100%|██████████| 30/30 [00:49<00:00,  1.64s/it]


In [11]:
for i in tqdm(list(set(puuid_list_master))):
    temp_list = riot.match_list(key, i)
    for i in temp_list:
        match_list_master.append(i)

100%|██████████| 203/203 [04:43<00:00,  1.39s/it]


In [12]:
matchids = pd.read_csv('matchids.csv')

In [13]:
match_info = pd.read_csv('match_info.csv')
match_info

,match_id,game_datetime,game_length,game_variation,game_version,tft_set_number
0,BR1_1925627632,2020-04-25 02:46:13,2436.686523,TFT3_GameVariation_FreeRerolls,Version 10.8.317.8137 (Apr 17 2020/16:09:11) [...,3


In [ ]:
for i in tqdm(matchids['matchid']):
    try:
        intermediate = riot.match_info(key, i)
        match_info = match_info.append(intermediate, ignore_index = True)
        match_info.drop_duplicates().to_csv('match_info.csv', index = False)
    except:
        pass
    
    time.sleep(0.5)

In [2]:
match_info.drop_duplicates().to_csv('match_info.csv', index = False)


NameError: name 'match_info' is not defined

In [10]:
match_info

,match_id,game_datetime,game_length,game_variation,game_version,tft_set_number
0,BR1_1925627632,2020-04-25 02:46:13,2436.686523,TFT3_GameVariation_FreeRerolls,Version 10.8.317.8137 (Apr 17 2020/16:09:11) [...,3
